### 1. Generate foggy datasets

Environment Setup

In [ ]:
# !pip install opencv-python numpy matplotlib scikit-image lpips tqdm
# !git clone https://github.com/DepthAnything/Depth-Anything-V2.git
# !pip install -r Depth-Anything-V2/requirements.txt

In [ ]:
# # Download pretrained model
# !mkdir -p Depth-Anything-V2/pretrained

# # Pretrained model 1: vit-base (middle precision)
# !wget https://huggingface.co/depth-anything/Depth-Anything-V2-Base/resolve/main/depth_anything_v2_vitb.pth
# !mv depth_anything_v2_vitb.pth Depth-Anything-V2/pretrained

Import Library

In [ ]:
# ============================================================
# 📦 1. 依存関係のインストール
# ============================================================
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os, sys, time, urllib.request, zipfile
from tqdm import tqdm
from glob import glob
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
import lpips
import torch

sys.path.append('/content/Depth-Anything-V2')

from depth_anything_v2.dpt import DepthAnythingV2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.5 MB/s eta 0:00:00


In [ ]:
encoder = 'vitb' # or 'vits', 'vitb', 'vitg'
DEPTH_MODEL_PATH = f'/content/Depth-Anything-V2/pretrained/depth_anything_v2_{encoder}.pth'
DEVICE = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'

model_configs = {
    'vits': {'encoder': 'vits', 'features': 64, 'out_channels': [48, 96, 192, 384]},
    'vitb': {'encoder': 'vitb', 'features': 128, 'out_channels': [96, 192, 384, 768]},
    'vitl': {'encoder': 'vitl', 'features': 256, 'out_channels': [256, 512, 1024, 1024]},
    'vitg': {'encoder': 'vitg', 'features': 384, 'out_channels': [1536, 1536, 1536, 1536]}
}

model = DepthAnythingV2(**model_configs[encoder])
model.load_state_dict(torch.load(DEPTH_MODEL_PATH, map_location=DEVICE))
model = model.to(DEVICE).eval()

In [ ]:
# ============================================================
# 📁 2. EuRoC MAV データセットのダウンロード
# ============================================================
# !curl -O http://robotics.ethz.ch/~asl-datasets/ijrr_euroc_mav_dataset/machine_hall/MH_04_difficult/MH_04_difficult.zip
# !unzip MH_04_difficult.zip > /dev/null 2>&1
# !rm -r MH_04_difficult.zip
# !mv mav0 MH_04_difficult

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  692M  100  692M    0     0  4922k      0  0:02:24  0:02:24 --:--:-- 3878k


In [ ]:
# ============================================================
# 🌤️ 3. 霧生成手法の実装
# ============================================================

# 霧を生成する関数
def asm_fog_basic(image, depth, beta=0.05, A=1.0):
    """
    霧のある画像を生成する
    :param image: 入力画像（理想的な画像）
    :param depth: 深度画像
    :param beta: 霧の濃さ（透過率に影響）
    :param A: 大気光の強度
    :return: 霧のある画像
    """
    # 画像サイズを取得
    h, w, _ = image.shape

    # 透過率 t(x) を計算
    depth = 1.0 - depth.mean(axis=2, keepdims=True) / depth.max()
    t = np.exp(-beta * depth)

    # 霧のある画像を生成
    foggy_image = image * t + A * (1 - t)

    # ピクセル値を0-255にクリップして整数型に変換
    foggy_image = foggy_image.astype(np.uint8)
    foggy_image = np.clip(foggy_image, 0, 255)

    return A*(1-t), foggy_image


# ============================================================
# 🌤️ 4. 霧除去手法の実装
# ============================================================

# --- 4.1. Dark Channel Prior (DCP) ---
def dehaze_DCP(image, omega=0.95, t0=0.1, win_size=15):
    # Convert image to float32 for computation
    img = np.array(image, dtype=np.float32) / 255.0

    # Compute dark channel
    dark = np.min(img, axis=2)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (win_size, win_size))
    dark = cv2.erode(dark, kernel)

    # Estimate atmospheric light
    A = np.percentile(img[dark >= np.percentile(dark, 99)], 99)

    # Estimate transmission map
    t = 1 - omega * (dark / A)
    t = np.clip(t, t0, 1)

    # Recover scene radiance
    J = np.zeros_like(img)
    for c in range(3):
        J[:, :, c] = (img[:, :, c] - A) / t + A

    # Clip and convert back to uint8 RGB
    J = np.clip(J * 255, 0, 255).astype(np.uint8)
    return J


# --- 4.2. Histogram Equalization (HE) ---
def dehaze_HE(image):
    # Convert RGB → YCrCb
    ycrcb = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
    y, cr, cb = cv2.split(ycrcb)

    # Equalize Y channel
    y_eq = cv2.equalizeHist(y)

    # Merge and convert back to RGB
    ycrcb_eq = cv2.merge((y_eq, cr, cb))
    image_eq = cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2RGB)

    return image_eq


# --- 4.3. CLAHE (Contrast Limited Adaptive Histogram Equalization) ---
def dehaze_CLAHE(image, clipLimit=6.0, tileGridSize=4):
    # Convert RGB → LAB
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)

    # Apply CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=(tileGridSize, tileGridSize))
    l_clahe = clahe.apply(l)

    # Merge and convert back to RGB
    lab_clahe = cv2.merge((l_clahe, a, b))
    image_clahe = cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2RGB)

    return image_clahe


In [ ]:
# ============================================================
# 📦 5. データセット作成クラス
# ============================================================

class DatasetCreator:
    def __init__(self, RAW_IMAGE_DIR, HAZY_IMAGE_DIR, DEHAZE_IMAGE_DIR=None, model=None,
                 dehaze_methods=['dcp', 'he', 'clahe']):
        self.raw_image_dir = RAW_IMAGE_DIR
        self.hazy_image_dir = HAZY_IMAGE_DIR
        self.dehaze_image_dir = DEHAZE_IMAGE_DIR
        self.model = model
        self.dehaze_methods = dehaze_methods

        self.raw_image_list = glob(f"{self.raw_image_dir}/*")
        os.makedirs(self.hazy_image_dir, exist_ok=True)
        if self.dehaze_image_dir:
            os.makedirs(self.dehaze_image_dir, exist_ok=True)

    # ------------------------------------------------------------
    # 霧画像生成
    # ------------------------------------------------------------
    def create_hazy_image(self, image, beta=0.05, A=1.0):
        """
        Creates hazy image using depth estimation.
        """
        if self.model is None:
            raise ValueError("Depth estimation model must be provided for haze synthesis.")

        depth = self.model.infer_image(image)
        depth = np.expand_dims(depth, axis=2) if depth.ndim == 2 else depth
        _, hazy_image = asm_fog_basic(image, depth, beta=beta, A=A)
        return hazy_image

    def create_hazy_dataset(self, beta=0.05, A=1.0, rename=False, suffix="png"):
        """
        Generates a directory of hazy images from clean images.
        """
        image_names = [f"hazy_{i}.{suffix}" if rename else os.path.basename(p)
                       for i, p in enumerate(self.raw_image_list)]

        for img_path, name in tqdm(zip(self.raw_image_list, image_names), total=len(self.raw_image_list)):
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            hazy_image = self.create_hazy_image(image, beta, A)
            save_path = os.path.join(self.hazy_image_dir, name)
            cv2.imwrite(save_path, cv2.cvtColor(hazy_image, cv2.COLOR_RGB2BGR))

        print(f"\n✅ Hazy dataset created successfully: {self.hazy_image_dir}")

    # ------------------------------------------------------------
    # 霧除去画像生成
    # ------------------------------------------------------------
    def create_dehaze_dataset(self):
        """
        Applies selected dehazing methods to the hazy dataset.
        """
        if not self.dehaze_image_dir:
            raise ValueError("DEHAZE_IMAGE_DIR must be specified to save dehazed images.")

        hazy_images = glob(f"{self.hazy_image_dir}/*")

        for hazy_path in tqdm(hazy_images, desc="Dehazing images"):
            image_name = os.path.basename(hazy_path)
            image = cv2.imread(hazy_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            for method in self.dehaze_methods:
                if method.lower() == 'dcp':
                    result = dehaze_DCP(image)
                elif method.lower() == 'he':
                    result = dehaze_HE(image)
                elif method.lower() == 'clahe':
                    result = dehaze_CLAHE(image)
                else:
                    print(f"⚠️ Unknown method: {method}")
                    continue

                method_dir = os.path.join(self.dehaze_image_dir, method)
                os.makedirs(method_dir, exist_ok=True)
                save_path = os.path.join(method_dir, image_name)
                cv2.imwrite(save_path, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))

        print(f"\n✅ Dehaze dataset created successfully: {self.dehaze_image_dir}")


# ============================================================
# 🎞️ 6. 画像群を動画に変換
# ============================================================

def images_to_video(image_folder, video_name, fps=30):
    """
    Converts a folder of images into a video.
    """
    images = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])
    if not images:
        raise ValueError(f"No images found in {image_folder}")

    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, _ = frame.shape

    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

    for image in tqdm(images, desc="Creating video"):
        frame = cv2.imread(os.path.join(image_folder, image))
        video.write(frame)

    video.release()
    print(f"🎬 Video saved as: {video_name}")

In [ ]:
import itertools
import os
from pathlib import Path

# ============================================================
# 🌫️ 霧条件設定
# ============================================================

betas = [1, 3, 5]
As = [30, 100, 230]

# === 全ての組み合わせを生成 ===
targets = [
    {"name": f"beta{beta}_A{A}", "beta": beta, "A": A}
    for beta, A in itertools.product(betas, As)
]

# ============================================================
# ⚙️ データセット生成ループ
# ============================================================

for target in targets:
    target_name = target["name"]
    beta = target["beta"]
    A = target["A"]

    # RAW, HAZY, DEHAZE ディレクトリ設定
    raw_dirs   = [f"/content/MH_04_difficult/cam{i}/data" for i in [0]]
    hazy_dirs  = [f"/content/HazyDatasets/{target_name}/hazy/cam{i}/data" for i in [0]]
    dehaze_dirs = [f"/content/HazyDatasets/{target_name}/dehazed/cam{i}/data" for i in [0]]

    video_name_hazy   = f"/content/HazyDatasets/{target_name}/hazy_video_cam0.mp4"
    video_name_dehaze = f"/content/HazyDatasets/{target_name}/dehaze_video_cam0.mp4"

    print(f"\n=== Generating dataset: {target_name} (β={beta}, A={A}) ===")

    # ============================================================
    # 🌫️ Step 1: Generate Hazy Dataset
    # ============================================================

    for j, (raw_dir, hazy_dir) in enumerate(zip(raw_dirs, hazy_dirs)):
        os.makedirs(hazy_dir, exist_ok=True)
        print(f"  -> Generating hazy images for camera {j} ...")

        creator = DatasetCreator(
            RAW_IMAGE_DIR=raw_dir,
            HAZY_IMAGE_DIR=hazy_dir,
            model=model  # ← 事前学習済みの深度推定モデルを指定
        )

        creator.create_hazy_dataset(beta=beta, A=A)

    print(f"  -> Creating hazy video: {video_name_hazy}")
    images_to_video(hazy_dirs[0], video_name_hazy, fps=30)

    # ============================================================
    # ☀️ Step 2: Dehaze (DCP / HE / CLAHE)
    # ============================================================

    for j, (hazy_dir, dehaze_dir) in enumerate(zip(hazy_dirs, dehaze_dirs)):
        os.makedirs(dehaze_dir, exist_ok=True)
        print(f"  -> Applying dehazing methods to camera {j} ...")

        creator = DatasetCreator(
            RAW_IMAGE_DIR=None,
            HAZY_IMAGE_DIR=hazy_dir,
            DEHAZE_IMAGE_DIR=dehaze_dir,
            dehaze_methods=['dcp', 'he', 'clahe']
        )

        creator.create_dehaze_dataset()

    # Optional: create video of one dehaze method (e.g., CLAHE)
    clahe_dir = os.path.join(dehaze_dirs[0], "clahe")
    if os.path.exists(clahe_dir):
        print(f"  -> Creating dehaze video: {video_name_dehaze}")
        images_to_video(clahe_dir, video_name_dehaze, fps=30)

print("\n✅ All hazy and dehazed datasets and videos generated successfully.")



=== Generating dataset: MH04_beta1_A30 (β=1, A=30) ===
  -> Generating hazy images for camera 0 ...


100%|██████████| 2033/2033 [05:39<00:00,  5.98it/s]



✅ Hazy dataset created successfully: /content/HazyDatasets/MH04_beta1_A30/hazy/cam0/data
  -> Creating hazy video: /content/HazyDatasets/MH04_beta1_A30/hazy_video_cam0.mp4


Creating video: 100%|██████████| 2033/2033 [00:21<00:00, 94.14it/s]


🎬 Video saved as: /content/HazyDatasets/MH04_beta1_A30/hazy_video_cam0.mp4
  -> Applying dehazing methods to camera 0 ...


...

🎬 Video saved as: /content/HazyDatasets/MH04_beta5_A230/hazy_video_cam0.mp4
  -> Applying dehazing methods to camera 0 ...


Dehazing images: 100%|██████████| 2033/2033 [02:22<00:00, 14.29it/s]



✅ Dehaze dataset created successfully: /content/HazyDatasets/MH04_beta5_A230/dehazed/cam0/data
  -> Creating dehaze video: /content/HazyDatasets/MH04_beta5_A230/dehaze_video_cam0.mp4


Creating video: 100%|██████████| 2033/2033 [00:18<00:00, 108.69it/s]

🎬 Video saved as: /content/HazyDatasets/MH04_beta5_A230/dehaze_video_cam0.mp4

✅ All hazy and dehazed datasets and videos generated successfully.


In [ ]:
# ============================================================
# 📈 Evaluation of Hazy / Dehazed Images vs. Clear Images
# ============================================================
import os
import cv2
import time
import torch
import lpips
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

# LPIPS metric (on GPU if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
loss_fn_alex = lpips.LPIPS(net='alex').to(device)

# Parameter sets
betas = [1, 3, 5]
As = [30, 230]

# Base directories
clear_dir = "/content/MH_04_difficult/cam0/data"
hazy_base = "/content/HazyDatasets"
dehaze_methods = ["clahe", "dcp", "he"]

# Prepare results
records = []

# ============================================================
# 🔁 Loop over all β, A conditions
# ============================================================
for beta in betas:
    for A in As:
        condition = f"MH04_beta{beta}_A{A}"
        hazy_dir = f"{hazy_base}/{condition}/hazy/cam0/data"

        if not os.path.exists(hazy_dir):
            print(f"⚠️ Skipped: {hazy_dir} not found.")
            continue

        img_files = sorted([f for f in os.listdir(hazy_dir) if f.endswith(".png")])
        if len(img_files) == 0:
            print(f"⚠️ No images in {hazy_dir}")
            continue

        print(f"🌀 Evaluating: {condition} ({len(img_files)} images)")

        # ------------------------------------------------------------
        # Compare HAZY vs CLEAR
        # ------------------------------------------------------------
        for f in tqdm(img_files, desc=f"Hazy vs Clear ({condition})"):
            ref_path = os.path.join(clear_dir, f)
            hazy_path = os.path.join(hazy_dir, f)
            if not os.path.exists(ref_path):
                continue

            ref = cv2.imread(ref_path)
            hazy = cv2.imread(hazy_path)

            psnr_val = psnr(ref, hazy)
            ssim_val = ssim(cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY),
                            cv2.cvtColor(hazy, cv2.COLOR_BGR2GRAY))

            # ✅ Convert to torch tensor for LPIPS
            ref_tensor = torch.from_numpy(
                cv2.cvtColor(ref, cv2.COLOR_BGR2RGB).transpose(2,0,1)[None,...] / 255.0
            ).float().to(device)
            hazy_tensor = torch.from_numpy(
                cv2.cvtColor(hazy, cv2.COLOR_BGR2RGB).transpose(2,0,1)[None,...] / 255.0
            ).float().to(device)

            with torch.no_grad():
                lpips_val = loss_fn_alex(ref_tensor, hazy_tensor).item()

            records.append([condition, "Hazy", f, psnr_val, ssim_val, lpips_val, 0.0])

        # ------------------------------------------------------------
        # Compare DEHAZED vs CLEAR
        # ------------------------------------------------------------
        for method in dehaze_methods:
            dehaze_dir = f"{hazy_base}/{condition}/dehazed/cam0/data/{method}"
            if not os.path.exists(dehaze_dir):
                print(f"⚠️ Skipped: {dehaze_dir} not found.")
                continue

            img_files = sorted([f for f in os.listdir(dehaze_dir) if f.endswith(".png")])
            for f in tqdm(img_files, desc=f"{method.upper()} vs Clear ({condition})"):
                ref_path = os.path.join(clear_dir, f)
                dehazed_path = os.path.join(dehaze_dir, f)
                if not os.path.exists(ref_path):
                    continue

                ref = cv2.imread(ref_path)
                dehazed = cv2.imread(dehazed_path)

                psnr_val = psnr(ref, dehazed)
                ssim_val = ssim(cv2.cvtColor(ref, cv2.COLOR_BGR2GRAY),
                                cv2.cvtColor(dehazed, cv2.COLOR_BGR2GRAY))

                ref_tensor = torch.from_numpy(
                    cv2.cvtColor(ref, cv2.COLOR_BGR2RGB).transpose(2,0,1)[None,...] / 255.0
                ).float().to(device)
                dehazed_tensor = torch.from_numpy(
                    cv2.cvtColor(dehazed, cv2.COLOR_BGR2RGB).transpose(2,0,1)[None,...] / 255.0
                ).float().to(device)

                with torch.no_grad():
                    lpips_val = loss_fn_alex(ref_tensor, dehazed_tensor).item()

                records.append([condition, method.upper(), f, psnr_val, ssim_val, lpips_val, 0.0])

# ============================================================
# 💾 Save results
# ============================================================
df = pd.DataFrame(records, columns=["Condition", "Method", "Image", "PSNR", "SSIM", "LPIPS", "Time[s]"])
os.makedirs("results", exist_ok=True)
df.to_csv("results/metrics.csv", index=False)
print("✅ Evaluation complete. Results saved to results/metrics.csv")

# ============================================================
# 📊 Visualization
# ============================================================
sns.boxplot(x="Method", y="PSNR", hue="Condition", data=df)
plt.title("PSNR Comparison (Hazy / Dehazed vs Clear)")
plt.xticks(rotation=30)
plt.show()


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.12/dist-packages/lpips/weights/v0.1/alex.pth
🌀 Evaluating: MH04_beta1_A30 (2033 images)


HE vs Clear (MH04_beta1_A30): 100%|██████████| 2033/2033 [01:57<00:00, 17.24it/s]


🌀 Evaluating: MH04_beta1_A230 (2033 images)


Hazy vs Clear (MH04_beta1_A230):  36%|███▌      | 722/2033 [00:41<01:16, 17.12it/s]